In [ ]:
# nuclio: start-code

In [ ]:
import sys, os
import ast
module_paths = ast.literal_eval(os.getenv('MODULE_PATHS', None))
if module_paths != None:
    for module_path in module_paths:
        sys.path.append(module_path)
        
import_modules = ast.literal_eval(os.getenv('IMPORT_MODULES', None))
if import_modules != None:
    imports={}
    import importlib
    for  module in import_modules:
        imports[module] = importlib.import_module(module)

In [ ]:
import ast
import json
import v3io.dataplane

In [ ]:
def igz_post_to_stream(context,message):
    Records=[]
    # message must be a Python dictionary
    message = json.dumps(message).encode('utf-8')
    Records.append({
            "data":  message,
            "partition_key": message['PartitionKey'],
            })
    context.user_data.v3io_client.put_records(container=os.getenv('OUTPUT_STREAM_CONTAINER'),
                                             path=os.getenv('OUTPUT_STREAM'),records=Records)


In [ ]:
def igz_stream_init(context):
    setattr(context.user_data,'v3io_client',
            v3io.dataplane.Client(endpoint='https://v3io-webapi:8081', 
                                  access_key=os.getenv("V3IO_ACCESS_KEY")))


In [ ]:
def process(context,message):
    # message is dictionary loaded from the stream event
    _function="context.user_data.processing_class.%s(context,message)"%os.getenv('PROCESSING_FUNCTION')
    post_message = eval(_function)
    if os.getenv('OUTPUT_STREAM') != None:
        igz_post_to_stream(context,post_message)
    
    if os.getenv('POST_PROCESS_FUNCTION') != None:
        _module, _function = os.getenv('POST_PROCESS_FUNCTION').split(".")
        _post_process_function = "imports['%s'].%s(context,message)"%(_module, _function)
        eval(_post_process_function)
    return

In [ ]:
def init_context(context):
    # IMPORTANTE: Load the model here
    # This code will put the model in the context of the function
    _module, _class = os.getenv('CLASS_LOAD_FUNCTION').split(".")
    _load_class = "imports['%s'].%s()"%(_module, _class)
    setattr(context.user_data,'processing_class',eval(_load_class))
    
    # Required for inference graph
    if os.getenv('OUTPUT_STREAM') != None:
        igz_stream_init(context)

    return
    

In [ ]:
def handler(context, event):
    context.logger.debug(event.body)
    try:
        message = ast.literal_eval(event.body.decode('utf-8'))
    except:
        print("Malformated Json ", event.body)
    message['shard']= event.shard_id
    process(context,message)
      
    return 

In [ ]:
# nuclio: end-code